In [1]:
!pip3 install -q -U transformers==4.38.2

In [2]:
import transformers
import os
from google.colab import userdata

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

**ALL IN ONE CODE** <br>


In [4]:
import pandas as pd
from transformers import AutoTokenizer

with open("/content/yungridataset.txt", 'r') as file:

    file_contents = file.readlines()

ques=[]
ans=[]
for line in file_contents:
   if line !="\n":
      if line.split()[0]=="Question:":
          ques.append(line.replace("\n",""))
      else:
          ans.append(line.replace("\n",""))

# remove Question: ques
f_que=[]
for i in ques:
  f_que.append(i.replace("Question:",""))

# remove Question: ques
f_ans=[]
for i in ans:
  f_ans.append(i.replace("Answer:",""))

#generailize variables ------------------------------------------------------->
model_id="google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
final_chat = []

for i in range(len(f_que)):
    que = {"role": "user", "content": f_que[i]}
    ans = {"role": "assistant", "content": f_ans[i]}
    final_chat.append(que)
    final_chat.append(ans)

chat_to_format=tokenizer.apply_chat_template(final_chat, tokenize=False)

c=chat_to_format.split("<end_of_turn>")
user_list=[]
model_list=[]
new_user_str=''
new_model_str=''


user_start_format='<start_of_turn>user'
model_start_format="<start_of_turn>model"
end_format='<end_of_turn>'


for i in range(len(c)-1):

  if i==0:  #000 part --start
      if c[i].split()[0][5:] == user_start_format and c[i].split()[0] != "" :
          user_q=c[i]+end_format
          user_list.append(user_q)
  else:    #000 part --end

      if c[i].split()[0] == user_start_format and c[i].split()[0] != "" :
              user_q=c[i]+end_format
              user_list.append(user_q)

      elif c[i].split()[0] == model_start_format and c[i].split()[0] != "" :
          model_a=c[i]+end_format
          model_list.append(model_a)
dataset=[]
dataset_str=''
for i in range(len(user_list)):
  dataset_str=user_list[i]+model_list[i]
  dataset.append(dataset_str)

df=pd.DataFrame(dataset)
df.head(10)

,0
0,<bos><start_of_turn>user\nWhat was yungri sart...
1,\n<start_of_turn>user\nWhere is the Yungri com...
2,\n<start_of_turn>user\nWho are the founders of...
3,\n<start_of_turn>user\nWhat type of products d...
4,\n<start_of_turn>user\nWhat materials are Yung...
5,\n<start_of_turn>user\nCan buyers customize th...
6,\n<start_of_turn>user\nHow can buyers contact ...
7,\n<start_of_turn>user\nWhat is the contact num...
8,\n<start_of_turn>user\nWhat is the email addre...
9,\n<start_of_turn>user\nWhere can buyers place ...


In [5]:
df.to_csv('yungrioutput.csv', index=False)